In [139]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from matplotlib import mlab
from scipy import signal, stats
from sklearn import preprocessing
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from datetime import datetime
from datetime import timedelta
import jpholiday

Tokyo_station_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Tokyostation/Tokyostation_2021.npy"
Tokyo_station_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Tokyostation_2021/outlier/Tokyostation_3zi_2021.npy"

Kyoto_station_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Kyotostation.npy"
Kyoto_station_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Kyotostation_users.npy"

Arashi_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Arashiyama_3zi_2022.npy"
Arashi_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Arashiyama_users.npy"

High_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Highclass_3zi_2022.npy"
High_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Highclass_users.npy"

Kinkaku_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Kinkaku_3zi_2022.npy"
Kinkaku_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Kinkaku_users.npy"

Kiyomizu_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Kiyomizu_3zi_2022.npy"
Kiyomizu_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Kiyomizu_users.npy"

Lowclass_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Lowclass_3zi_2022.npy"
Lowclass_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Lowclass_users.npy"

Nizyou_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Nizyou_3zi_2022.npy"
Nizyou_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Nizyou_users.npy"

Touzi_mobile_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/mobile/Kyoto/Touzi_3zi_2022.npy"
Touzi_tweets_PATH = "/home/is/shuntaro-o/dev/compare_population_and_tweet_number/data/twitter/Kyoto/users/Touzi_users.npy"

list_mobile = [
    Tokyo_station_mobile_PATH,
    Arashi_mobile_PATH,
    High_mobile_PATH,
    Kinkaku_mobile_PATH,
    Kiyomizu_mobile_PATH,
    Lowclass_mobile_PATH,
    Nizyou_mobile_PATH,
    Touzi_mobile_PATH,
    Kyoto_station_mobile_PATH,
]
list_twitter = [
    Tokyo_station_tweets_PATH,
    Arashi_tweets_PATH,
    High_tweets_PATH,
    Kinkaku_tweets_PATH,
    Kiyomizu_tweets_PATH,
    Lowclass_tweets_PATH,
    Nizyou_tweets_PATH,
    Touzi_tweets_PATH,
    Kyoto_station_tweets_PATH,
]
list_key = [
    "東京駅",
    "嵐山",
    "中京区",
    "金閣寺",
    "清水寺",
    "学生街",
    "二条城",
    "東寺",
    "京都駅",
]

In [140]:
def isBizDay(DATE):
    # DATE = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    if DATE.weekday() >= 5 or jpholiday.is_holiday(DATE):
        return "Holiday"
    else:
        return "Workday"

In [147]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        data.append([str(i) + '時', correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
fig.update_layout(title='1時間ごとのソーシャルセンサ性能 (平日)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='時間'),
                 yaxis=dict(title='ソーシャルセンサ性能'))
fig.show()

In [148]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
fig.update_layout(title='1時間ごとのソーシャルセンサ性能 (平日, p値>0.05を削除)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='時間'),
                 yaxis=dict(title='ソーシャルセンサ性能'))
fig.show()

In [149]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        data.append([str(i) + '時', correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
fig.update_layout(title='1時間ごとのソーシャルセンサ性能 (休日)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='時間'),
                 yaxis=dict(title='ソーシャルセンサ性能'))
fig.show()

In [150]:
data = []
for place_id in range(0, len(list_mobile)):
    mobile = np.load(list_mobile[place_id])
    tweets = np.load(list_twitter[place_id])
    dlete_list = []
    if place_id == 0:
        start_date = datetime(2021,1,1)
    else:
        start_date = datetime(2022,1,1)

    for n_date_from_year in range(0,365):
        Today = start_date + timedelta(days=n_date_from_year)
        work_holi_Flag = isBizDay(Today)
        if (
            n_date_from_year == 0
            or n_date_from_year == 1
            or n_date_from_year == 2
            or n_date_from_year == 362
            or n_date_from_year == 363
            or n_date_from_year == 364
            or work_holi_Flag == "Holiday"
            ):
            continue
        else:
            dlete_list.append(n_date_from_year)
    mobile = np.delete(mobile,dlete_list,axis=0)
    tweets = np.delete(tweets,dlete_list,axis=0)

    for i in range(0,24):
        mobile_1hour = mobile[:, i:i+1]
        mobile_1hour = mobile_1hour.flatten()
        tweets_1hour = tweets[:, i:i+1]
        tweets_1hour = tweets_1hour.flatten()

        correlation, p_value = stats.pearsonr(mobile_1hour, tweets_1hour)
        if p_value>0.05:
            data.append([i,'', p_value, list_key[place_id]])
        else:
            data.append([i, correlation, p_value, list_key[place_id]])


columns=['time','r','p','place']


df= pd.DataFrame(data,columns=columns)



list_place = df['place'].unique()
fig = go.Figure(data=[
    go.Scatter(x = df[df['place']==list_place[0]]['time'], y = df[df['place']==list_place[0]]['r'], name = list_place[0],),
    go.Scatter(x = df[df['place']==list_place[1]]['time'], y = df[df['place']==list_place[1]]['r'], name = list_place[1],),
    go.Scatter(x = df[df['place']==list_place[2]]['time'], y = df[df['place']==list_place[2]]['r'], name = list_place[2],),
    go.Scatter(x = df[df['place']==list_place[3]]['time'], y = df[df['place']==list_place[3]]['r'], name = list_place[3],),
    go.Scatter(x = df[df['place']==list_place[4]]['time'], y = df[df['place']==list_place[4]]['r'], name = list_place[4],),
    go.Scatter(x = df[df['place']==list_place[5]]['time'], y = df[df['place']==list_place[5]]['r'], name = list_place[5],),
    go.Scatter(x = df[df['place']==list_place[6]]['time'], y = df[df['place']==list_place[6]]['r'], name = list_place[6],),
    go.Scatter(x = df[df['place']==list_place[7]]['time'], y = df[df['place']==list_place[7]]['r'], name = list_place[7],),
    go.Scatter(x = df[df['place']==list_place[8]]['time'], y = df[df['place']==list_place[8]]['r'], name = list_place[8],),
    ])
fig.update_layout(title='1時間ごとのソーシャルセンサ性能 (休日, p値>0.05を削除)')
fig.update_layout(yaxis=dict(range=(-1, 1)))
fig.update_layout(xaxis=dict(title='時間'),
                 yaxis=dict(title='ソーシャルセンサ性能'))